In [1]:
import rispy
import bibtexparser
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# jupyter nbconvert --to notebook --execute Codes.ipynb --output=new.ipynb

In [2]:
def parse_title(title):
    title = title.strip()
    title = re.sub('^[\s\{\"]*','',title)
    title = re.sub('[\s\}\"]$','',title)
    title = re.sub('\~$','?',title) # this is an issue here, because both ? and : are translated to ~
    title = title.replace("’", "'")
    title = title.replace("~", ":") 
    title = re.sub('\w+\s\:\s','',title)
    return title

def get_bibID_from_title(title, bib_file_path):
    bibtex = bibtexparser.load(open(bib_file_path, 'r'))
    
    for entry in bibtex.entries:
        if title.lower() == parse_title(entry['title']).lower():
            return entry["ID"] # to fail if ID not present
    raise ValueError(f'Report title not found on bibliogaphy: [{title}] ')
    
def remove_code_flag(code):
    new_code =  re.sub("\s+\((conclusion|driver)\)\s*$", '', code)
    new_code = new_code.strip()
    return new_code

In [3]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [4]:
# NVIVO Windows
df_nvivo = pd.read_excel('../reports/nvivo_codes_win.xlsx')

# NVIVO MAC
# df_nvivo = pd.read_csv('../reports/nvivo_codes_mac.csv')


In [5]:
studies_titles = [parse_title(title).split('\\')[-1] for title in df_nvivo.columns[1:]]
studies_citations = [get_bibID_from_title(title, '../bibliography/references.bib') for title in studies_titles ]

pd.DataFrame(data= {'Title': studies_titles,'ID': studies_citations}).sort_values(by=['Title'],ascending=True).set_index('ID')


Title
ID                                                                                                                                                             
Park2021                                                                                                    7s model for technology protection of organizations
Ncubukezi2020                                                                        A Review of the Current Cyber Hygiene in Small and Medium-sized Businesses
McLaurin2021                                                                                   A Study on the Efficacy of Small Business Cybersecurity Controls
Huaman2021                                      A large-scale interview study on information security in and attacks against small and medium-sized enterprises
Mokwetli2018                                                               Adoption of the ICT Security Culture in SMME's in the Gauteng Province, South Africa
Eilts2020                                                                 An Empirical Assessment of Cybersecurity Readiness and Resilience in Small Businesses
White2020                                             Antecedents of Cybersecurity Implementation: A Study of the Cyber-Preparedness of U.K. Social Enterprises
Barlette2017                                                                                CEOs' information security behavior in SMEs: Does ownership matter?
Madison2017                                           Challenges with Funding of Information Security Projects per Threat Perceptions: A Qualitative Case Study
Heidenreich2019                     Conceptualization of a Measurement Method Proposal for the Assessment of IT Security in the Status Quo of Micro-Enterprises
Rawindaran2021                                 Cost Benefits of Using Machine Learning Features in NIDS for Cyber Security in UK Small Medium Enterprises (SME)
Loffler2021                                                                              CySecEscape 2.0—A Virtual Escape Room To Raise Cybersecurity Awareness
Carias2021                                                                                              Cyber Resilience Self-Assessment Tool (CR-SAT) for SMEs
FeltonJr.2021                                                                                                         Cyber Resilience of Small Business Owners
Lejaka2019                                                             Cyber security awareness for small, medium and micro enterprises (SMMEs) in South Africa
Orellana2020                                          Cybersecurity Incident Response Capabilities in the Ecuadorian Small Business Sector: A Qualitative Study
Alahmari2020a                                       Cybersecurity Risk Management in Small and Medium-Sized Enterprises: A Systematic Review of Recent Evidence
Dykstra2020a                                                                        Cybersecurity in Medical Private Practice: Results of a Survey in Audiology
Mierzwa2017                                                                                      Cybersecurity in non-profit and non-governmental organizations
Ozkan2019                                                          Cybersecurity standardisation for SMEs: The stakeholders' perspectives and a research agenda
Feher2020                                              Cybersecurity threats of cloud and third-party services in small and medium-sized enterprise environment
Saa2017                                                                                           Data security issues in cloud-based Software-as-a-Service ERP
Elezaj2019                                                                              Data-Driven Intrusion Detection System for Small and Medium Enterprises
Rae2019                                                                               Defining a New Composite Cybersecurity Rating Scheme for SMEs in the

In [6]:
df_parsed = df_nvivo.set_axis(['Codes', *studies_citations],axis=1, copy=True)

# removing hirearchical names (only windows has full names)
df_parsed['Codes'] = df_parsed['Codes'].apply(lambda x: x.replace("Codes\\\\FullText Codes\Organized Codes\\", ""))


df_parsed.insert(1, "Count", None)
df_parsed.insert(2, "Studies", None)


for idx in df_parsed.index:
    studies = []
    
    df_parsed.loc[idx, 'Codes'] = parse_title(df_parsed.loc[idx, 'Codes'])
    
    for col in df_parsed.columns[3:]:
        cel = df_parsed.loc[idx, col]            
        
        if type(cel) != np.int64:
            print (f'something is wrong with cell col=[{col}] and row [{idx}]')
            
        else:
            if df_parsed.loc[idx, col] > 0:
                studies.append(f'[@{col}]')
            
    df_parsed.loc[idx,'Studies'] = ' '.join(studies)
    df_parsed.loc[idx,'Count'] = len(studies)
    

In [7]:
df_parsed = df_parsed[["Codes", "Count", "Studies"]].sort_values(by=['Codes'],ascending=True)

df_parsed.to_csv('../tables/codes-child.csv', encoding='utf-8', quoting=0 ,sep=",",   index=False)

df_parsed




Codes  \
0                                                                                                              Automation and DYI maximize investments in cybersecurity (conclusion)   
1                                                                                                                     Cloud adoption minimises cybersecurity challenges (conclusion)   
2                                                               Cloud adoption minimises cybersecurity challenges (conclusion)\Cloud adoption shifted threats landscape (conclusion)   
3    Cloud adoption minimises cybersecurity challenges (conclusion)\Cloud adoption shifted threats landscape (conclusion)\Misconception of shared responsibility models (conclusion)   
4                                                                                  Cloud adoption minimises cybersecurity challenges (conclusion)\Improves availability (conclusion)   
5                                                                                     Cloud adoption minimises cybersecurity challenges (conclusion)\Lower investiments (conclusion)   
6                                                                                  Cloud adoption minimises cybersecurity challenges (conclusion)\Lower risk perception (conclusion)   
7                                                                                                                         Cloud adoption minimises cybersecurity challenges (driver)   
8                                                                       Cloud adoption minimises cybersecurity challenges (driver)\Cloud adoption shifted threats landscape (driver)   
9                                                                                Cloud adoption minimises cybersecurity challenges (driver)\Increase access to technologies (driver)   
10                                                                                             Cloud adoption minimises cybersecurity challenges (driver)\Lower investments (driver)   
11                                                                                         Cloud adoption minimises cybersecurity challenges (driver)\Lower risk perception (driver)   
12                                                                                                                                            Cloud adoption resistance (conclusion)   
13                                                                                                                                                Cloud adoption resistance (driver)   
14                                                                                                                      Compliance driven efforts do not raise security (conclusion)   
15                                                                                                                                     Covid accelerated cloud adoption (conclusion)   
16                                                                                                                                         Covid accelerated cloud adoption (driver)   
17                                                                                                                                   Covid accelerated cybersecurity Issues (driver)   
18                                                                                                                                      Covid drove cybersecurity awareness (driver)   
19                                                                                                                             Education does not improve cybersecurity (conclusion)   
20                                                                                                                            Employee rewarding or gamification system (conclusion)   
21                                                                                                                             

In [8]:
df_codes = pd.DataFrame( columns=['Codes', 'Driver' 'Conclusion', 'StudiesDriver', 'StudiesConclusion'], index=['Codes'])
df_parsed = df_parsed.sort_values(by=['Codes'],ascending=False)


list_of_parsed_themes = {}
for idx in df_parsed.index:
    code = df_parsed.loc[idx, 'Codes']
    code_parsed = remove_code_flag(code)
    
    
    theme_dic = list_of_parsed_themes.get(code_parsed)
    if not theme_dic: 
        theme_dic = {
            'Code':code_parsed, 
            'Driver':0, 
            'Conclusion':0,
            'Total' : 0,
            'StudiesDriver' : '',
            'StudiesConclusion' : ''
        }
        list_of_parsed_themes[code_parsed] = theme_dic
        
    
#     number_of_studies = len(df_win3.loc[idx, 'Studies'])
    number_of_studies = df_parsed.loc[idx, 'Studies'].count("@")
    if '(conclusion)' in code: 
        theme_dic['Conclusion'] = number_of_studies
        theme_dic['StudiesConclusion'] = df_parsed.loc[idx, 'Studies']
        
    elif '(driver)' in code: 
        theme_dic['Driver'] = number_of_studies
        theme_dic['StudiesDriver'] = df_parsed.loc[idx, 'Studies']
    
    elif '(driver & conclusion)' in code: 
        theme_dic['Driver'] = theme_dic['Conclusion'] = theme_dic['Total'] = number_of_studies
        theme_dic['StudiesConclusion'] = df_parsed.loc[idx, 'Studies']
        theme_dic['StudiesDriver'] = df_parsed.loc[idx, 'Studies']
        
    else:  
        print(f'CODE [{code}] is netiher driver nor conclusion')

    
#     counting the absolute number of studies, removing duplications between driver and conclusion.
    studies_absolute_list = theme_dic['StudiesDriver'].split(' ') + theme_dic['StudiesConclusion'].split(' ')
    studies_absolute_list = list( dict.fromkeys(studies_absolute_list)) # dedup
    studies_absolute_list = list(filter(None, studies_absolute_list)) # remove empty entries
    theme_dic['Total'] =  len(studies_absolute_list)
        
    

In [9]:
df_codes = pd.DataFrame(list(list_of_parsed_themes.values()),columns = ['Code', 'Driver', 'Conclusion', 'Total', 'StudiesDriver', 'StudiesConclusion']) 

df_codes.sort_values(by=['Code'],ascending=True,inplace=True)

df_codes

Code  \
132                                                                                                            Automation and DYI maximize investments in cybersecurity   
126                                                                                                                   Cloud adoption minimises cybersecurity challenges   
131                                                             Cloud adoption minimises cybersecurity challenges (conclusion)\Cloud adoption shifted threats landscape   
130  Cloud adoption minimises cybersecurity challenges (conclusion)\Cloud adoption shifted threats landscape (conclusion)\Misconception of shared responsibility models   
129                                                                                Cloud adoption minimises cybersecurity challenges (conclusion)\Improves availability   
128                                                                                   Cloud adoption minimises cybersecurity challenges (conclusion)\Lower investiments   
127                                                                                Cloud adoption minimises cybersecurity challenges (conclusion)\Lower risk perception   
125                                                                 Cloud adoption minimises cybersecurity challenges (driver)\Cloud adoption shifted threats landscape   
124                                                                          Cloud adoption minimises cybersecurity challenges (driver)\Increase access to technologies   
123                                                                                        Cloud adoption minimises cybersecurity challenges (driver)\Lower investments   
122                                                                                    Cloud adoption minimises cybersecurity challenges (driver)\Lower risk perception   
121                                                                                                                                           Cloud adoption resistance   
120                                                                                                                     Compliance driven efforts do not raise security   
119                                                                                                                                    Covid accelerated cloud adoption   
118                                                                                                                              Covid accelerated cybersecurity Issues   
117                                                                                                                                 Covid drove cybersecurity awareness   
116                                                                                                                            Education does not improve cybersecurity   
115                                                                                                                           Employee rewarding or gamification system   
114                                                                                                                                                       Faced threats   
113                                                                                                                              Faced threats (driver)\Insider Threats   
112                                                                                                                                               Forced digitalization   
111                                                                                                                         Forced digitalization (driver)\Competitions   
110                                                                                                                Implementation challenges lowers security commitment   
109                                                        

In [10]:


df_codes = df_codes.sort_values(by=['Total'], ascending=False)

df_codes

Code  \
101                                                                                                                                                   Lack of awareness   
84                                                                                                                                                     Limited literacy   
8                                                                                                          Under-funded and resource-constrained cybersecurity programs   
77                                                                                                                            Literature overlooked SMEs specific needs   
91                                                                                                                            Lack of tailored solutions and frameworks   
61                                                                                                                                                  Low risk perception   
54                                                                                                         Overwhelmed, sparse or non-existent cybersecurity leadership   
32                                                                                                                                      Rising cyberattacks against SME   
109                                                                                                                      Increasing financial loss due to cyber-attacks   
126                                                                                                                   Cloud adoption minimises cybersecurity challenges   
46                                                                                                                                             Poor security operations   
58                                                                                                    Low risk perception (driver)\Less susceptible because of the size   
30                                                                                                                       Risk awareness improve cybersecurity behaviour   
87                                                                                                            Legislation and informative content improve cybersecurity   
86                                                                                                                                Legislation gap and lack of standards   
112                                                                                                                                               Forced digitalization   
76                                                                                       Literature overlooked SMEs specific needs (driver)\Actionable insights for SME   
64                                                                                                Low risk perception (conclusion)\Less susceptible because of the size   
16                                                                                                                             Tailored services and solutions do exist   
19                                                                                                                                                  Supply-chain effect   
65                                                                                                                        Low risk perception (conclusion)\From Leaders   
44                                                                                                     Poor security operations (driver)\Uncontrolled use of BYOD & IoT   
98                                                                                                                           Lack of awareness (driver)\From Leadership   
41                                                         

In [11]:

# csv.QUOTE_* values. Acceptable values are 0, 1, 2, and 3 for QUOTE_MINIMAL, QUOTE_ALL, QUOTE_NONE, and QUOTE_NONNUMERIC,

# saving CSV with Markwdown synthax
# df_codes.to_csv('../tables/codes-parsed-md.csv', encoding='utf-8',  sep=",",  index=False)

# saving CSV with Latex synthax

for col in ['StudiesDriver','StudiesConclusion']:
    df_codes[col] = df_codes[col].apply(lambda x: x.replace("[@", "\\cite{"))
    df_codes[col] = df_codes[col].apply(lambda x: x.replace("]", "}"))
    
df_codes['Code'] = df_codes['Code'].apply(lambda x: x.replace("&", "and"))
df_codes['Code'] = df_codes['Code'].apply(lambda x: x.replace(",", " "))

df_codes = df_codes.sort_values(by=['Code'], ascending=True)
df_codes.to_csv('../tables/codes-parsed-latex-alpha.csv', encoding='utf-8', sep=",",  index=False)

df_codes = df_codes[~df_codes['Code'].str.contains("\\\\")] # filtering out child nodes
df_codes = df_codes.sort_values(by=['Total'], ascending=False)
df_codes.to_csv('../tables/codes-parsed-latex.csv', encoding='utf-8', sep=",",  index=False)

# TODO consider migrate tables to https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_latex.html



In [12]:
# df_chart = df_codes.query('Total > 5').sort_values(by=['Total'],ascending=True)
# ylabels = df_chart[["Studies"]].to_numpy()

# df_chart.plot.barh(x="Code", y="Total", rot=70, title="Studies per Theme")
# # plt.yticks(ticks =np.arange(len(ylabels)) , rotation=0,  labels=ylabels)
# plt.yticks(rotation=0, ticks =np.arange(len(ylabels)), )
# plt.savefig('../images/codes', dpi=300, bbox_inches='tight')
# plt.show(block=True)
